# Hidden Markov Models

Outline:
1. Markov model
1. State transition matrix
1. Maximum likelihood parameter assignment
1. Hidden Markov Model
1. Probability of observed sequence
1. Viterbi algorithms


## Example

<img src=images/hmm1.png height=400/>


Hidden states = {rainy, sunny}
observables = {walk, shop, clean}

## 1. Markov model

Given set of states $S = \{s_1, ..., s_m\}$ we observe series over time $x = \{x_1, ..., x_T\}$, $x \in S^T$

Assumptions about markov model:
* Limited horizon  
$P(x_t | x_{t-1}, x_{t-2}, ..., x_{t-n}) = P(x_{t} | x_{t-1})$

* Stationary process  
$P(x_{t} | x_{t-1}) = P(x_2 | x_1)$ for $t \in 2..T$

## 2 State transition matrix 

State transition matrix $A \in R^{|S| x |S|}$, where  
$A_{ij}$ is probabilty of transition from state $s_i$ to state $s_j$

We compute probability of the particular sequence x by chain rule using limited horizon assumption:  
$$P(x_T, x_{T-1}, x_{T-2}, ..., x_1; A) = \\
= P(x_T | x_{T-1}, ..., x_0; A)P(x_{T-1} | x_{T-2}, ..., x_0; A)...P(x_1 | x_0; A)P(x_0;A)\\ 
 =P(x_{T} | x_{T-1}; A) P(x_{T-1} | x_{T-2}; A) P(x_2 | x_1; A) P(x_1 | x_0; A) = \\ = \prod_{T=1}^T P(x_{T} | x_{T-1}; A) = \prod_{T=1}^T A_{x_{T-1}, x_t}$$

## 3 Maximum likelihood parameter assignment

$$Likelihood(A) = log P(z; A) = \\ 
= log \prod_{t=1}^T A_{z_{t-1}, z_t} = \sum_{t=1}^T log A_{z_{t-1}, z_t} = \\ 
= \sum_{t=1}^T \sum_{i=1}^{|S|} \sum_{j=1}^{|S|} [z_{t-1} = s_i \wedge z_t = s_j]log A_{ij}$$

We want:  
* $l(A) \rightarrow \underset {A} {\max}$  
* $\sum_{j=1}^{|S|} A_{ij} = 1 $  
* $A_{ij} \geq 0$  

With Lagrange multipliers we can get following estimate:  
$$\hat A_{ij} = \frac {\sum_{t=1}^T  [z_{t-1} = s_i \wedge z_t = s_j]} {\sum_{t=1}^T  [z_{t-1} = s_i]}$$

## 4 Hidden Markov Model

<img src=images/hmm2.png height=400/>

Let   
$S$ - set of hidden states  
$O$ - set of observables  
$x_t \in S$ - hidden variables  
$y_t \in O$ - observed variables  
$A \in R^{|S|x|S|}$ - transition matrix  
$B \in R^{|S|x|O|}$ - emition matrix  

Probability of sequence of observed states:

$$P(y; A, B) = \sum_{x_0, .., x_T} P(y, x; A, B) = \sum_{x_0, .., x_T} P(y|x; A, B) P(x; A, B) = \\ 
= \sum_{x_0, .., x_T} ( \prod_{t=1}^T P(y_t|x_t; B) ) ( \prod_{t=1}^T P(x_t|x_{t-1}; A) ) = \\ 
= \sum_{x_0, .., x_T} ( \prod_{t=1}^T B_{x_t, y_t} ) ( \prod_{t=1}^T A_{x_{t-1}, x_t} )$$  


Evaluating the prob directly costs $O(|S|^T)$

Fundamental questions for HMM:
* What is the probability of the observed sequence?  
Given HMM (A, B) and observations x, caclulate probability that HMM generated x.
* What is the most likely series of states to generate the observations?  
Given HMM (A, B) and observations x, caclulate the most likely sequence of hidden states, that produced observations x.  
* How can we learn A and B?  
Given some training observations x and general structure of HMM (number of hidden and visible states), determine (A, B) that best fit the data.

##  5 Probability of observed sequence

###Forward algorithm 
for computing for computing probability of observed sequence:

Define $$\alpha_i(t) = P(y_1, y_2, ..., y_t, x_t=s_i; A, B)$$ - total probability of all observations up through time t, and being at state s_i at time t.  
Then,  
$$P(x; A, B) = P(y_1, ..., y_T; A, B) = \sum_{i=1}^{|S|} P(y_1, ..., y_T, x_T = s_i; A, B)) = \sum_{i=1}^{|S|} \alpha_i(T)$$

We can compute with for $O(|S| T)$ by dynamic programming:  
$\alpha_i (0) = A_{0, i} $  
$\alpha_j (t) = \sum_{i=1}^{|S|} \alpha_i (t-1) A_{ij} B_{j, y_t} $


## 6 Viterbi algorithm  

For maximum likelihood state assignment. 

Given series of outputs $y \in O^T$:  

$$arg max_x P(x|y; A, B) = arg max_x \frac {P(y, x; A, B)} {\sum_x P(y, x; A, B)} = arg max_x P(y, x; A, B)$$

Naive approach in $O(|S|^T)$.  

Let $\pi[j, s]$ - max probability for any state sequence ending in state $s$ at position $t=j$.

$$\pi[1, s] = A_{0, s} B_{s, y_1}$$

$$\pi[j, s] = max_{i \in {1 .. k}} \pi[j-1, i] A_{i, s} B_{s, y_j}$$
$$bp[j, s] = arg max_{i \in {1 .. k}} \pi[j-1, i] A_{i, s} B_{s, y_j}$$

Recover all sequence:  
$$s_T = argmax_s \pi[T, s]$$  
$$s_{j-1} = bp[j, s_j]$$

Complexity $O(T |S|^2)$